In [1]:
!pip install mlflow optuna dagshub imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import dagshub
import mlflow
dagshub.init(repo_owner='AMR-ITH', repo_name='yt-comment-analyzer', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=ca10f1d0-129a-456a-b477-6e96092acb5b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=6e4c18fb48dc67d4184a2ec1209ccb1a798d50fc355790452e6d9e8a79c871e3




Accessing as AMR-ITH

Initialized MLflow to track repo "AMR-ITH/yt-comment-analyzer"

Repository AMR-ITH/yt-comment-analyzer initialized!

In [3]:
# Set or create an experiment
mlflow.set_experiment("ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/6372c8dc1c894e3290997ce5e2b55158', creation_time=1751625225992, experiment_id='4', last_update_time=1751625225992, lifecycle_stage='active', name='ML Algos with HP Tuning', tags={}>

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
import mlflow
import mlflow.sklearn
import optuna
from imblearn.under_sampling import RandomUnderSampler

In [6]:
df = pd.read_csv("/content/reddit_preprocessing.csv").dropna()
df.head()

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1
2,seriously say thing first get complex explain ...,-1
3,learned want teach different focus goal not wr...,0
4,benefit may want read living buddha living chr...,1


In [ ]:
df.shape

(36662, 2)

In [7]:
# Step 1: Clean data
df = df.dropna(subset=['category'])
y = df['category']
X_raw = df['clean_comment']

# Step 2: Train-test split BEFORE vectorization or resampling
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Vectorization only on training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=3000)
X_train_vect = vectorizer.fit_transform(X_train_raw)
X_test_vect = vectorizer.transform(X_test_raw)  # Note: transform only!

# Step 4: Undersampling (only on training data)
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_vect, y_train)

# # Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_train_resampled, y_train_resampled, test_size=0.2, random_state=42, stratify=y_train_resampled)


# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_RandomUnderSampler_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)



# Step 6: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)  # Tuning the smoothing parameter

    # MultinomialNB model setup
    model = MultinomialNB(alpha=alpha)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Multinomial Naive Bayes, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])

    # Log the best model with MLflow, passing the algo_name as "MultinomialNB"
    log_mlflow("MultinomialNB", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Multinomial Naive Bayes
run_optuna_experiment()


[I 2025-07-11 12:53:38,135] A new study created in memory with name: no-name-781ba7f6-8766-4041-a0b5-79362db2eb47
[I 2025-07-11 12:53:38,147] Trial 0 finished with value: 0.6865370042940137 and parameters: {'alpha': 0.37343829413960705}. Best is trial 0 with value: 0.6865370042940137.
[I 2025-07-11 12:53:38,157] Trial 1 finished with value: 0.6910836069714574 and parameters: {'alpha': 0.0007245287849870212}. Best is trial 1 with value: 0.6910836069714574.
[I 2025-07-11 12:53:38,166] Trial 2 finished with value: 0.6880525385198283 and parameters: {'alpha': 0.2956051436255813}. Best is trial 1 with value: 0.6910836069714574.
[I 2025-07-11 12:53:38,176] Trial 3 finished with value: 0.6819904016165699 and parameters: {'alpha': 0.6312360667980088}. Best is trial 1 with value: 0.6910836069714574.
[I 2025-07-11 12:53:38,186] Trial 4 finished with value: 0.6915887850467289 and parameters: {'alpha': 0.1212668459724741}. Best is trial 4 with value: 0.6915887850467289.
[I 2025-07-11 12:53:38,194]

🏃 View run MultinomialNB_RandomUnderSampler_TFIDF_Trigrams at: https://dagshub.com/AMR-ITH/yt-comment-analyzer.mlflow/#/experiments/4/runs/289b7e1f03544d858caf9d7d2518dad9
🧪 View experiment at: https://dagshub.com/AMR-ITH/yt-comment-analyzer.mlflow/#/experiments/4
